In [1]:
import pandas as pd
import json
import minsearch

# Ingestion

In [152]:
with open('../data/arsonor_chunks_300_50.json', 'r', encoding='utf-8') as file:
    documents = json.load(file)

In [153]:
documents[15]

{'article_id': '3632a3b4',
 'url': 'https://arsonor.com/lintelligence-artificielle-ia-dans-le-studio-de-production-audio-5-6/',
 'title': 'L’intelligence artificielle (IA) dans le studio de production audio (5/6)',
 'category': 'LA POST-PROD',
 'tags': 'dé-mixage, de-noise, de-reverb, deep learning, plug-in audio, restauration audio, stems',
 'chunk_id': '3632a3b4-11',
 'chunk_text': "Ce plugin, de part sa simplicité d’utilisation (un ou deux potards à régler) est vite devenu une référence absolue, un « must-have » auprès de tout les ingénieurs du son d’un studio de musique. Plug-in Drumatom de Accusonus Accusonus Regroover pour séparer les éléments d'une boucle de batterie Un autre de leurs logiciels remarquables est Regroover . Celui-ci permet de décomposer efficacement une boucle audio complexe (souvent rythmique, de batterie) en plusieurs boucles contenant chacune un élément/instrument séparé de la boucle principale! Très utile par exemple pour isoler le kick, la snare, les high ha

### With Minsearch:

In [111]:
index = minsearch.Index(
    text_fields=['title', 'tags', 'chunk_text'],
    keyword_fields=['article_id', 'category', 'chunk_id']
)

In [112]:
index.fit(documents)

### With Elasticsearch

In [154]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import json

In [155]:
es = Elasticsearch("http://localhost:9200")

In [200]:
index_name = "arsonor_chunks_300"

# Create index if not already created
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body={
        "mappings": {
            "properties": {
                "article_id": {"type": "keyword"},
                "title": {"type": "text"},
                "url": {"type": "keyword"},
                "category": {"type": "keyword"},
                "tags": {"type": "text"},
                "chunk_id": {"type": "keyword"},
                "chunk_text": {"type": "text"},
                # "embedding": {"type": "dense_vector", "dims": 768}  # assuming 768-dim embeddings
            }
        }
    })

In [201]:
def prepare_documents_for_indexing(docs):
    for doc in docs:
        yield {
            "_index": index_name,
            "_id": doc['chunk_id'],
            "_source": {
                "article_id": doc['article_id'],
                "title": doc['title'],
                "url": doc['url'],
                "category": doc['category'],
                "tags": doc['tags'],
                "chunk_id": doc['chunk_id'],
                "chunk_text": doc['chunk_text'],
                # "embedding": generate_embedding(doc['chunk_text'])  # Add embedding here
            }
        }

# Index the documents in bulk
bulk(es, prepare_documents_for_indexing(documents))

(572, [])

# RAG flow

In [158]:
from openai import OpenAI
client = OpenAI()

In [175]:
query = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'

In [12]:
response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": query}]
    )
    
response.choices[0].message.content

"Pour créer de la musique dans un home-studio, voici une liste du matériel nécessaire :\n\n### 1. **Ordinateur**\n   - Un ordinateur performant (PC ou Mac) avec une bonne quantité de RAM et un processeur rapide pour faire tourner des logiciels de production musicale.\n\n### 2. **Logiciel de Production Musicale (DAW)**\n   - Logiciels tels que Ableton Live, FL Studio, Logic Pro, Pro Tools, ou Reaper pour l'enregistrement, l'édition et la production de musique.\n\n### 3. **Interfaces Audio**\n   - Une interface audio pour connecter vos instruments et microphones à votre ordinateur avec une meilleure qualité sonore.\n\n### 4. **Moniteurs de Studio**\n   - Des enceintes de monitoring pour écouter votre musique avec précision. Choisissez des modèles adaptés à la taille de votre pièce.\n\n### 5. **Casque Audio**\n   - Un bon casque de studio pour le mixage et l'enregistrement, offrant une isolation phonique et une réponse plate.\n\n### 6. **Clavier MIDI**\n   - Un clavier MIDI pour jouer des

### Search results with Minsearch

In [62]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

### Search results with Elasticsearch

In [217]:
def elastic_search(query, category):
    search_query = {
        "size": 10,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["title", "tags", "chunk_text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "category": category
                    }
                }
            }
        }
    }

    response = es.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

### Prompt

In [207]:
prompt_template = """
You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally, recommend the top 3 Arsonor articles (refer to their 'title') that are the best to read for answering this question.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

entry_template = """
article title: {title}
article keywords: {tags}
content: {chunk_text}
""".strip()


In [230]:
def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context += entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context)
    return prompt

In [222]:
search_results = elastic_search(query, category='LE HOME STUDIO')
prompt = build_prompt(query, search_results)

In [223]:
print(prompt)

You're an audio engineer and sound designer instructor for beginners.
You're particularly specialized in audio home-studio set-up, computer music production and audio post-production in general (editing, mixing and mastering). 
Answer the QUESTION based on the CONTEXT from our arsonor knowledge database (articles).
Use only the facts from the CONTEXT when answering the QUESTION.
Finally, recommend the top 3 Arsonor articles (refer to their 'title') that are the best to read for answering this question.

QUESTION: De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?

CONTEXT:
article title: Deviens toi aussi producteur musical depuis ton home studio
article keywords: home-studio, ingénieur du son, MAO, production musicale
content: Avis à tous les passionnés de musique, c’est avec grand plaisir que j’ouvre ce blog afin de partager ma passion de la MAO (Musique Assistée par Ordinateur) et du home studio , de la création au mixage audio. Alors tu te demandes sûrement… 

In [17]:
def llm(prompt, model='gpt-4o-mini'):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
def rag(query, category=None, model='gpt-4o-mini'):
    search_results = elastic_search(query, category)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [65]:
category = 'LE HOME STUDIO'
question = 'De quel matériel ai-je besoin pour créer ma musique dans mon home-studio?'
answer = rag(question)
print(answer)

Pour créer ta musique dans un home-studio, tu auras besoin des éléments suivants :

1. **Ordinateur** : Un ordinateur récent, que ce soit un Mac ou un PC, capable de gérer des applications audio. Il doit avoir un processeur rapide, une carte mère avec de la capacité pour ajouter de la RAM, et suffisamment d'espace de disque.

2. **DAW (Digital Audio Workstation)** : C'est le logiciel principal que tu utiliseras pour enregistrer, jouer et créer ta musique. Les options populaires incluent Ableton Live, ProTools, Logic Pro, et bien d'autres. La DAW te permet d'éditer des fichiers audio et de manipuler les sons en suivant une ligne de temps.

3. **Casque audio** : Un casque de bonne qualité est essentiel pour commencer à écouter et affiner ton son, sans nécessiter tout de suite des enceintes de monitoring.

4. **Interface audio (optionnelle mais recommandée à long terme)** : Bien que la plupart des ordinateurs aient une carte son intégrée, une interface audio externe rendra l'enregistremen

# Retrieval evaluation

In [148]:
df_question = pd.read_csv('../data/ground-truth-300.csv')
df_question.head()

,question,category,chunk,article
0,Quel est l'impact de l'IA sur la post-producti...,LA POST-PROD,4615db39-1,4615db39
1,Comment les outils IA simplifient-ils le trava...,LA POST-PROD,4615db39-1,4615db39
2,Quels avantages l'IA apporte-t-elle aux artist...,LA POST-PROD,4615db39-1,4615db39
3,Comment un débutant peut-il améliorer ses prod...,LA POST-PROD,4615db39-1,4615db39
4,Quelle est l'évolution des outils audio pour l...,LA POST-PROD,4615db39-1,4615db39


In [149]:
ground_truth = df_question.to_dict(orient='records')
ground_truth[0]

{'question': "Quel est l'impact de l'IA sur la post-production audio et musicale",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [22]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

### 1) Basic chunking, Minsearch based on the chunk_id

In [45]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [24]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['chunk']
        results = search_function(q)
        relevance = [d['chunk_id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [25]:
from tqdm.auto import tqdm

In [26]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.5297113752122241, 'mrr': 0.3079710027757569}

### 2) Basic chunking, Minsearch based on the article_id

In [27]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['article']
        results = search_function(q)
        
        # Ensure that only the first relevant result counts for the MRR
        relevance = []
        seen_article = False
        for d in results:
            if d['article_id'] == doc_id and not seen_article:
                relevance.append(True)
                seen_article = True
            else:
                relevance.append(False)

        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }


In [28]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2945 [00:00<?, ?it/s]

{'hit_rate': 0.5911714770797962, 'mrr': 0.4871826070552721}

### 3) Find the best boost parameters

In [115]:
df_validation = df_question[:100]
df_test = df_question[100:]

In [116]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [117]:
gt_val = df_validation.to_dict(orient='records')

In [118]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [119]:
param_ranges = {
    'title': (0.0, 3.0),
    'tags': (0.0, 3.0),
    'chunk_text': (0.0, 3.0)
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [123]:
simple_optimize(param_ranges, objective, n_iterations=20)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

...

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

({'title': 0.3144003830483091,
  'tags': 0.21750500549018636,
  'chunk_text': 1.55775214448781},
 0.8056071428571429)

In [124]:
def minsearch_improved(query):
    boost = {
        'title': 0.31,
        'tags': 0.22,
        'chunk_text': 1.56
    }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

# evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

### 4) Dynamic chunking size 300, overlap 50 - Minsearch, text search

In [101]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2860 [00:00<?, ?it/s]

{'hit_rate': 0.5814685314685315, 'mrr': 0.47083888333888235}

In [108]:
evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/2860 [00:00<?, ?it/s]

{'hit_rate': 0.9545454545454546, 'mrr': 0.7798329448329442}

### 5) Dynamic chunking size 350, overlap 30 - Minsearch, text search

In [68]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

  0%|          | 0/2215 [00:00<?, ?it/s]

{'hit_rate': 0.6045146726862303, 'mrr': 0.4794655845784513}

In [125]:
evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

  0%|          | 0/2215 [00:00<?, ?it/s]

{'hit_rate': 0.8939051918735892, 'mrr': 0.7163696298685003}

### 6) Dynamic chunking - Elasticsearch, text search

chunk size 300, overlap 50:

In [150]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['category']))

  0%|          | 0/2860 [00:00<?, ?it/s]

{'hit_rate': 0.9629370629370629, 'mrr': 0.8279434454434451}

chunk size 350, overlap 30:

In [142]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['category']))

  0%|          | 0/2215 [00:00<?, ?it/s]

{'hit_rate': 0.9616252821670429, 'mrr': 0.7977245691354039}

# RAG evaluation

In [83]:
prompt2_template = """
You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [84]:
len(ground_truth)

2945

In [85]:
record = ground_truth[0]
record

{'question': "Quel est l'impact de l'intelligence artificielle dans la post-production audio?",
 'category': 'LA POST-PROD',
 'chunk': '4615db39-1',
 'article': '4615db39'}

In [87]:
question = record['question']
answer_llm = rag(question)

'L\'impact de l\'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l\'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.\n\nPremièrement, l\'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de leur travail. Par exemple, des plugins d\'IA peuvent s\'occuper de tâches telles que le nettoyage et la séparation des pistes audio de manière rapide et efficace, ce qui allège la charge de travail des ingénieurs du son. Des technologies avancées comme le dé-mixage, où une piste audio est décomposée en différentes stems (voix, batterie, etc.), sont déjà courantes et en constante amélior

In [88]:
print(answer_llm)

L'impact de l'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.

Premièrement, l'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de leur travail. Par exemple, des plugins d'IA peuvent s'occuper de tâches telles que le nettoyage et la séparation des pistes audio de manière rapide et efficace, ce qui allège la charge de travail des ingénieurs du son. Des technologies avancées comme le dé-mixage, où une piste audio est décomposée en différentes stems (voix, batterie, etc.), sont déjà courantes et en constante amélioration grâ

In [89]:
prompt = prompt2_template.format(question=question, answer_llm=answer_llm)
print(prompt)

You are an expert evaluator for a RAG system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: Quel est l'impact de l'intelligence artificielle dans la post-production audio?
Generated Answer: L'impact de l'intelligence artificielle (IA) dans la post-production audio est considérable et se manifeste par une amélioration des outils et processus disponibles pour les artistes, producteurs et ingénieurs du son. La démocratisation de l'accès à des technologies avancées permet à un plus grand nombre de créateurs de produire des œuvres de qualité professionnelle sans nécessiter une formation approfondie en ingénierie du son.

Premièrement, l'IA facilite la réalisation des tâches complexes et répétitives, permettant ainsi aux artistes et producteurs de se concentrer sur les aspects créatifs de 

In [90]:
import json

In [91]:
df_sample = df_question.sample(n=200, random_state=1)

In [92]:
sample = df_sample.to_dict(orient='records')

In [93]:
evaluations = []

for record in tqdm(sample):
    question = record['question']
    answer_llm = rag(question) 

    prompt = prompt2_template.format(
        question=question,
        answer_llm=answer_llm
    )

    evaluation = llm(prompt)
    evaluation = json.loads(evaluation)

    evaluations.append((record, answer_llm, evaluation))

  0%|          | 0/200 [00:00<?, ?it/s]

In [94]:
evaluations[0]

({'question': "Pourquoi le 32 bits float ne suffit pas avant l'enregistrement dans la DAW ?",
  'category': 'LE HOME STUDIO',
  'chunk': 'e55c4a41-6',
  'article': 'e55c4a41'},
 'Le format 32 bits float est très utile pour le traitement audio au sein d\'une station de travail audionumérique (DAW) car il offre une large plage dynamique et permet d’éviter le clipping pendant les manipulations internes. Cependant, il ne suffit pas avant l\'enregistrement dans la DAW pour plusieurs raisons.\n\nTout d\'abord, l\'avantage du 32 bits float ne s\'applique pas à la phase de l\'enregistrement, qui se déroule encore dans le domaine analogique. Pendant cette étape, il est crucial de s\'assurer que le signal audio ne dépasse pas le 0 dBFS pour éviter la saturation. De plus, une gestion rigoureuse des niveaux est essentielle dès le moment de l\'enregistrement pour maintenir une bonne qualité audio et éviter des problèmes de bruit de quantification lors des conversions numériques.\n\nEnsuite, une foi

In [95]:
df_eval = pd.DataFrame(evaluations, columns=['record', 'answer', 'evaluation'])

df_eval['id'] = df_eval.record.apply(lambda d: d['chunk'])
df_eval['question'] = df_eval.record.apply(lambda d: d['question'])

df_eval['relevance'] = df_eval.evaluation.apply(lambda d: d['Relevance'])
df_eval['explanation'] = df_eval.evaluation.apply(lambda d: d['Explanation'])

del df_eval['record']
del df_eval['evaluation']

In [96]:
df_eval.relevance.value_counts(normalize=True)

relevance
RELEVANT           0.985
PARTLY_RELEVANT    0.015
Name: proportion, dtype: float64

In [97]:
df_eval.to_csv('../data/rag-eval-gpt-4o-mini.csv', index=False)

In [99]:

df_eval[df_eval.relevance == 'PARTLY_RELEVANT']

,answer,id,question,relevance,explanation
18,Pour changer le pitch naturellement dans Ablet...,173567a9-2,Quelle méthode peut-on utiliser pour changer l...,PARTLY_RELEVANT,The generated answer discusses changing pitch ...
31,Pour éviter les pièges du make-up gain lors de...,584d0437-3,Quels conseils professionnels peut on suivre p...,PARTLY_RELEVANT,The generated answer discusses make-up gain an...
146,La question concernant l'activation du mode Th...,173567a9-11,Quelles sont les étapes pour activer le mode T...,PARTLY_RELEVANT,The generated answer acknowledges the question...
